In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!pip install geopandas
!pip install rasterio
!pip install attrdict
!pip install utm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 19.1 MB/s 
     |████████████████████████████████| 6.3 MB 45.0 MB/s 
     |████████████████████████████████| 16.7 MB 348 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6108 sha256=38b91bb0d054826bad6b3fe441fc237de9b56694d818d92cd8fa6341e4b920ff
  Stored in directory: /root/.cache/pip/wheels/a5/b0/12/7ee4fdb0f9fbb4157100bd02390436ed5d58ebfd3c6d6a0886
Successfully built utm


In [1]:
from osgeo import gdal
from attrdict import AttrDict
import utm
from pyproj import CRS
import os
from pprint import pprint

def get_resolution(filename):
    '''
    obtains the pixel size in meters of passed raster file

    Args:
        filename(str): name of raster file to be investigated
    '''

    tiffile = filename.split('/')[-1]
    dummypath = filename.split('/')[:-1]
    dummypath = '/'.join(dummypath)

    info = gdal.Info(filename, format='json')

    epsg = int(info['coordinateSystem']['wkt'].split('"EPSG","', 1)[-1].split('"')[0])

    if epsg != 4326:
        dummyfile = os.path.join(dummypath, 'epsgdummy.tif')
        warp = gdal.Warp(dummyfile, filename, dstSRS='EPSG:4326')
        warp = None 
        filename = dummyfile
        info = gdal.Info(filename, format='json')

    # print(f'initial epsg: {epsg}')

    info = AttrDict(info)

    center = info.cornerCoordinates.center
    if center[1] > 0:
        south = True
    else:
        south = False
    
    as_utm = utm.from_latlon(center[1], center[0])
    
    crs = CRS.from_dict({'proj': 'utm', 'zone': as_utm[2], 'south': south})

    epsg = crs.to_authority()

    utmpath = os.path.join(duke_path, 'sudan', 'lab', 'dummy.tif')
    utmpath = os.path.join(dummypath, 'utmdummy.tif')

    warp = gdal.Warp(utmpath, filename, dstSRS=epsg[0]+':'+epsg[1])
    warp = None 

    info = AttrDict(gdal.Info(utmpath, format='json'))
    pixel_size = info.geoTransform[1]

    os.remove(utmpath)
    try:
        os.remove(dummyfile)
    except UnboundLocalError:
        pass

    return pixel_size



base_path = '/content/drive/MyDrive/PyPSA_Africa_images/'
duke_path = os.path.join(base_path, 'duke')
countries = os.listdir(os.path.join(duke_path))

# imgs = os.path.join(base_path, 'maxar', 'chad', 'raw')

for country in countries:
    files = os.listdir(os.path.join(duke_path, country, 'raw'))
    files = [file for file in files if file.endswith('.tif') and 'multiclass' not in file]
    files = [file for file in files if os.stat(os.path.join(duke_path, country, 'raw', file)).st_size > 1_000_000]

    filename = os.path.join(duke_path, country, 'raw', files[0])

    resolution = get_resolution(filename)
    
    print(f'Resolution of {country}: {resolution} m/pixel')


Resolution of arizona: 0.1521273311113449 m/pixel
Resolution of sudan: 0.2941356391155026 m/pixel
Resolution of china: 0.2789294695267606 m/pixel
Resolution of rotorua: 0.1254966590856217 m/pixel
Resolution of brazil: 0.4787769055150278 m/pixel
Resolution of hartford: 0.0761820560981825 m/pixel
Resolution of mexico: 0.1522336654793647 m/pixel
Resolution of kansas: 0.1520310125331718 m/pixel
Resolution of clyde: 0.1535755375821943 m/pixel
Resolution of wilmington: 0.1522415121091316 m/pixel
Resolution of dunedin: 0.1229411434163525 m/pixel
Resolution of gisborne: 0.1253881937589445 m/pixel
Resolution of palmertson: 0.1255661866324052 m/pixel
Resolution of tauranga: 0.1254967811385344 m/pixel
